# Warehouse Problem - Lagrangian relaxation

Let us recall the original MIP formulation:

\begin{align*}
\min &\qquad \sum_w c_w x_w + \sum_{w ,c} t_{w,c} y_{w,c}  & \\
\text{subject to:} &&\\
&y_{w,c} \leq x_w & \forall w,c \\
&\sum_w y_{w,c} = 1 & \forall c \\
&\sum_c y_{w,c} \leq C_w  x_w & \forall w \\
&x_w, y_{w,c}, z_c \in \mathbb{B} & \forall w,c
\end{align*}

In [2]:
fixed = 30  # c_w (all the costs are the same)
M = 10*fixed  # the penalty
capacity = [1,4,2,1,3]  # the capacity of a warehouse
supplyCost = [[20,24,11,25,30],  # t_{w,c}
              [28,27,82,83,74],
              [74,97,71,96,70],
              [2,55,73,69,61],
              [46,96,59,83,4],
              [42,22,29,67,59],
              [1,5,73,59,56],
              [10,73,13,43,96],
              [93,35,63,85,46],
              [47,65,55,71,95]]
nbStores = len(supplyCost)
nbWarehouses = len(capacity)
Stores = range(nbStores)
Warehouses = range(nbWarehouses)

### Excercices

#### 1. Find the lagrangian relaxation formulation where the constraints 2 are relaxed.

#### 2. Implement the lagrangian relaxation.

In [3]:
import gurobipy as gp

def lagrangian_relaxation(lagrange_multipliers=[1] * nbStores):
    mdl = gp.Model(name='lagrangian')

    # Variables
    warehouse_is_open = mdl.addVars(nbWarehouses, vtype=gp.GRB.BINARY, name='warehouse_is_open')
    warehouse_supplies_store = mdl.addVars(Warehouses, Stores, vtype=gp.GRB.BINARY, name='warehouse_supplies_store')

    # Constraint
    only_open_warehouses_can_serve_stores = mdl.addConstrs(warehouse_supplies_store[w, c] <= warehouse_is_open[w] for w in Warehouses for c in Stores)
    # This is the constraint that we're going to relax.
    # each_store_is_served_by_one_warehouse = mdl.addConstrs(gp.quicksum(warehouse_supplies_store[w, c] for w in Warehouses) == 1 for c in Stores)

    warehouse_capacity_is_respected = mdl.addConstrs(gp.quicksum(warehouse_supplies_store[w, c] for c in Stores) <= capacity[w]*warehouse_is_open[w] for w in Warehouses)


    # Objective function
    cost_of_opening_a_warehouse = gp.quicksum(fixed*warehouse_is_open[w] for w in Warehouses)
    transportation_cost = gp.quicksum(supplyCost[c][w]*warehouse_supplies_store[w, c] for w in Warehouses for c in Stores)
    lagrange_cost = gp.quicksum(lagrange_multipliers[c] * (1 - gp.quicksum(warehouse_supplies_store[w, c] for w in Warehouses)) for c in Stores)

    mdl.setObjective(cost_of_opening_a_warehouse + transportation_cost + lagrange_cost, sense=gp.GRB.MINIMIZE)

    mdl.optimize()

    msol = (
        {key: var.X for key, var in warehouse_is_open.items()},
        {key: var.X for key, var in warehouse_supplies_store.items()},
    )

    relaxed_constraints = [1 - gp.quicksum(warehouse_supplies_store[w, c].X for w in Warehouses) for c in Stores]
    
    return (mdl.getObjective().getValue(),  # the objective value
            (cost_of_opening_a_warehouse.getValue(), transportation_cost.getValue(), lagrange_cost.getValue()),  # the cost of opening a warehouse, the transportation cost, the lagrange cost
            msol, # solution
            relaxed_constraints)  # slack of the relaxed constraints

#### Try the function

In [5]:
from pprint import pprint

(obj, (cost_of_opening_a_warehouse, transportation_cost, lagrange_cost), msol, relaxed) = lagrangian_relaxation([fixed]*nbStores)
pprint(msol)
pprint(relaxed)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.5.0 23F79)
Gurobi Compute Server Worker version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8175M CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 55 rows, 55 columns and 155 nonzeros
Model fingerprint: 0x8dfd205b
Variable types: 0 continuous, 55 integer (55 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 300.0000000
Presolve removed 51 rows and 51 columns
Presolve time: 0.00s
Presolved: 4 rows, 4 columns, 10 nonzeros
Found heuristic solution: objective 288.0000000
Variable types: 0 continuous, 4 integer (4 binary)

Root relaxation: objective 2.820000e+02, 0 iterations, 0.00 seconds (0.00 work units)

    N

#### 3. Implement the Cutting Plane Method

1. State the master problem
2. Keep track of the best feasible solution
3. Use the solution to the master problem to solve the Lagrangian subproblem

In [7]:
import numpy as np

rlambda = [fixed]*nbStores
fsol = None
iteration = 0
upper_bounds = []
cutting_planes = []
bestFeasibleValue = UB
bestFeasible = 0

master = gp.Model("master")
w = master.addVar(name='w', vtype=gp.GRB.CONTINUOUS)
master.setObjective(w, sense=gp.GRB.MAXIMIZE)
lagrange_multipliers = master.addVars(Stores, name='lagrange_multipliers')

cutting_planes.append(master.addConstr(1e10 >= w))

master.optimize()

lagrange_multipliers_values = [0] * nbStores

while not np.isclose(w.X, obj):
    # TODO: implement the loop

    (
        obj,  # the objective value
        (cost_of_opening_a_warehouse, transportation_cost, lagrange_cost),
        (warehouse_is_open, warehouse_supplies_store), # solution
        relaxed_constraints,  # slack of the relaxed constraints
    ) = lagrangian_relaxation(lagrange_multipliers_values)


    # Constraints
    expression = (cost_of_opening_a_warehouse + transportation_cost) + gp.quicksum(lagrange_multipliers[c] * relaxed_constraints[c] for c in Stores)
    cutting_planes.append(master.addConstr(expression >= w))

    master.optimize()

    lagrange_multipliers_values = [var.X for var in lagrange_multipliers.values()]

    print(f"Value of the Lagrange multipliers: {lagrange_multipliers_values}")
    print(f"Objective value of the subproblem: {obj}")
    print(f"Value of the Lagrangian dual: {w.X}")

    iteration += 1


print('========================')
print('========================')
print('========================')
print(f'Lagrangian dual: {w.X}')
print(f'Objective value of the subproblem: {obj}')
print(f'Optimal values of the Lagrange multipliers: {lagrange_multipliers_values}')
print(f'Number of iterations: {iteration}')
if bestFeasible != 0:
    print('-- best feasible --')
    print(bestFeasible)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.5.0 23F79)
Gurobi Compute Server Worker version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8175M CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1 rows, 11 columns and 1 nonzeros
Model fingerprint: 0xd4bceef8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+10, 1e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1 rows and 11 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+10   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optim